In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
physical_devices
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1,28*28).astype("float32") / 255.0

In [70]:
x_train.shape

(60000, 784)

In [96]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b", shape=(self.units,), initializer="random_normal", trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [87]:
class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU,self).__init__()
        
    def call(self,inputs):
        return tf.math.maximum(inputs,0)

In [107]:
class MyModel(keras.Model):
    def __init__(self, num_class):
        super(MyModel,self).__init__()
        self.dense1 = MyDense(64)
        self.dense2 = MyDense(num_class)
        self.relu = MyReLU()
        
    def call(self, input_tensor, training= False):
        x = self.dense1(input_tensor)
        x = self.relu(x)
        x = self.dense2(x)
        return x
    
    def model(self):
        x = layers.InputLayer(input_shape = (28*28))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [108]:
model = MyModel(10)

In [103]:
model = keras.Sequential(
    [Dense(32), MyReLU(), Dense(10)]
)

In [109]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'],
    optimizer = keras.optimizers.Adam()
)

In [110]:
model.fit(x_train, y_train, batch_size = 64)

938/938 [==============================] - 2s 2ms/step - loss: 0.4101 - accuracy: 0.8916


In [111]:
model.summary()

Model: "my_model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_49 (MyDense)        multiple                  50240     
_________________________________________________________________
my_dense_50 (MyDense)        multiple                  650       
_________________________________________________________________
my_re_lu_18 (MyReLU)         multiple                  0         
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
